In [1]:
using Pkg; Pkg.activate(".");
using Convex
using SCS

  Activating new project at `~/Dropbox (MIT)/IAP22/6.S098-project`


In [41]:
class_numbers = Dict("6.006"=>1, "6.042"=>2, "6.009"=>3, "6.046"=>4, "6.031"=>5, "18.600"=>6, "6.041"=>7,
"6.008"=>8, "comp_bio"=>9, "NLP"=>10, "comp_vision"=>11, "graphics"=>12, "optimization"=>13, 
    "script_analysis"=>14, "intro_to_acting"=>15, "psych"=>16)

taken = [1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 1]

requirements = [
  ([1], 1),
  ([2], 1),
  ([3], 1),
  ([4], 1),
  ([5], 1),
  ([6, 7, 8], 1),
  ([9, 10, 11, 12, 13], 2),
  ([14, 15, 16], 2)
];

In [42]:
remaining_requirements = []

for (classes, req) in requirements
  classes_left::Array{Int64} = []
  for class in classes
    if taken[class] == 1
      req = max(req - 1, 0)
      classes_left
    else
      push!(classes_left, class)
    end
  end
  if length(classes_left) > 0
    push!(remaining_requirements, (classes_left, req))
  end
end

remaining_requirements

5-element Vector{Any}:
 ([4], 1)
 ([5], 1)
 ([7, 8], 0)
 ([9, 10, 11, 13], 1)
 ([14, 15], 1)

In [43]:
class_names = Dict()
for (name, num) in class_numbers
  class_names[num] = name
end
class_names

Dict{Any, Any} with 16 entries:
  5  => "6.031"
  16 => "psych"
  12 => "graphics"
  8  => "6.008"
  1  => "6.006"
  6  => "18.600"
  11 => "comp_vision"
  9  => "comp_bio"
  14 => "script_analysis"
  3  => "6.009"
  7  => "6.041"
  13 => "optimization"
  15 => "intro_to_acting"
  4  => "6.046"
  2  => "6.042"
  10 => "NLP"

In [44]:
num_classes = length(class_names)
choices = Variable(num_classes)

Variable
size: (16, 1)
sign: real
vexity: affine
id: 131…674

In [45]:
ratings = [
  1,
  0,
  0.5,
  0.25,
  0.01,
  1,
  0,
  0,
  1,
  0.5,
  0.49,
  0,
  0,
  1,
  0,
  0
];

In [46]:
# general 1 unit per class
class_units = ones(num_classes)
obj = - ratings'*choices + class_units'*choices

+ (affine; real)
├─ * (affine; real)
│  ├─ 1×16 adjoint(::Vector{Float64}) with eltype Float64
│  └─ 16-element real variable (id: 131…674)
└─ * (affine; real)
   ├─ 1×16 adjoint(::Vector{Float64}) with eltype Float64
   └─ 16-element real variable (id: 131…674)

In [55]:
# max units allowed (set arbitrarily)
max_units = sum([j for (i, j) in remaining_requirements]) + 1
min_units = 2

consts = [
  #relaxation of binary choice
  choices >= 0,
  choices <= 1,
  # max units
  sum(choices) <= max_units,
  # min units
  sum(choices) >= min_units,
]

# classes already taken cannot be taken again
for (class, done) in enumerate(taken)
  consts += choices[class] == 0
end

# class requirements
for (classes, req) in remaining_requirements
  req_choices = 0
  for class in classes
    req_choices += choices[class]
  end
  consts += req_choices >= req
end

consts

25-element Vector{Constraint}:
 >= constraint (affine)
├─ 16-element real variable (id: 131…674)
└─ 0
 <= constraint (affine)
├─ 16-element real variable (id: 131…674)
└─ 1
 <= constraint (affine)
├─ sum (affine; real)
│  └─ 16-element real variable (id: 131…674)
└─ 5
 >= constraint (affine)
├─ sum (affine; real)
│  └─ 16-element real variable (id: 131…674)
└─ 2
 == constraint (affine)
├─ index (affine; real)
│  └─ 16-element real variable (id: 131…674)
└─ 0
 == constraint (affine)
├─ index (affine; real)
│  └─ 16-element real variable (id: 131…674)
└─ 0
 == constraint (affine)
├─ index (affine; real)
│  └─ 16-element real variable (id: 131…674)
└─ 0
 == constraint (affine)
├─ index (affine; real)
│  └─ 16-element real variable (id: 131…674)
└─ 0
 == constraint (affine)
├─ index (affine; real)
│  └─ 16-element real variable (id: 131…674)
└─ 0
 == constraint (affine)
├─ index (affine; real)
│  └─ 16-element real variable (id: 131…674)
└─ 0
 ⋮
 == constraint (affine)
├─ index (affine; re

In [56]:
problem = minimize(obj, consts)
solve!(problem, SCS.Optimizer())

------------------------------------------------------------------
	       SCS v3.2.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 17, constraints m: 56
cones: 	  z: primal zero / dual free vars: 17
	  l: linear vars: 39
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 103, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 5.00e+00  1.00e+00  1.04e+00  3.32e-01  1.00e-01  9.27e-05 
    50| 5.13e+06  3.90e+05  2.12e+19  1.06e+19  1.00e-01  1.65e-04 
------------------------------------

┌ Warning: Problem status INFEASIBLE; solution may be inaccurate.
└ @ Convex /Users/timkostolansky/.julia/packages/Convex/ukggP/src/solution.jl:342
